# Task 1

Bayesian Gaussian classifier

In [2]:
import numpy as np

np.random.seed(0)

# исходные параметры распределений двух классов
r1 = 0.7
D1 = 1.0
mean1 = [1, -2]
V1 = [[D1, D1 * r1], [D1 * r1, D1]]

r2 = -0.5
D2 = 2.0
mean2 = [0, 2]
V2 = [[D2, D2 * r2], [D2 * r2, D2]]

# моделирование обучающей выборки
N1 = 500
N2 = 1000
x1 = np.random.multivariate_normal(mean1, V1, N1).T
x2 = np.random.multivariate_normal(mean2, V2, N2).T

data_x = np.hstack([x1, x2]).T
data_y = np.hstack([np.ones(N1) * -1, np.ones(N2)])

# вычисление оценок МО и ковариационных матриц
mm1 = np.mean(x1.T, axis=0)
mm2 = np.mean(x2.T, axis=0)

a = (x1.T - mm1).T
VV1 = np.array([[np.dot(a[0], a[0]) / N1, np.dot(a[0], a[1]) / N1],
                [np.dot(a[1], a[0]) / N1, np.dot(a[1], a[1]) / N1]])

a = (x2.T - mm2).T
VV2 = np.array([[np.dot(a[0], a[0]) / N2, np.dot(a[0], a[1]) / N2],
                [np.dot(a[1], a[0]) / N2, np.dot(a[1], a[1]) / N2]])

# для гауссовского байесовского классификатора
Py1, L1 = 0.5, 1  # вероятности появления классов
Py2, L2 = 1 - Py1, 1  # и величины штрафов неверной классификации

func = lambda x, lm, py, m, cov: np.log(lm * py) - 1/2 * (x - m).T @ np.linalg.inv(cov) @ (x - m) - 1/2 * np.log(np.linalg.det(cov))
predict = []
classes = [-1, 1]
TP = TN = FP = FN = 0
for i in range(len(data_x)):
    a = np.argmax([func(data_x[i], L1, Py1, mm1, VV1), func(data_x[i], L2, Py2, mm2, VV2)]) # return indexes (0 or 1)
    predict.append(classes[a])
    if classes[a] == 1 and data_y[i] == 1:
        TP += 1
    elif classes[a] == -1 and data_y[i] == -1:
        TN += 1
    elif classes[a] == 1 and data_y[i] == -1:
        FP += 1
    elif classes[a] == -1 and data_y[i] == 1:
        FN += 1
print(TP, TN, FP, FN)

947 491 9 53


# Task 2

Stochastic Gradient Descent

In [14]:
import numpy as np

# логарифмическая функция потерь
def loss(w, x, y):
	margin = np.dot(w, x) * y
	return np.log2(1 + np.exp(-margin))

# производная логарифмической функции потерь по вектору w
def df(w, x, y):
	margin = np.dot(w, x) * y
	num = np.exp(-margin) * x.T * y 
	den = np.log(2) * (1 + np.exp(-margin))
	return - num / den

data_x = [(5.8, 1.2), (5.6, 1.5), (6.5, 1.5), (6.1, 1.3), (6.4, 1.3), (7.7, 2.0), (6.0, 1.8), (5.6, 1.3), (6.0, 1.6), (5.8, 1.9), (5.7, 2.0), (6.3, 1.5), (6.2, 1.8), (7.7, 2.3), (5.8, 1.2), (6.3, 1.8), (6.0, 1.0), (6.2, 1.3), (5.7, 1.3), (6.3, 1.9), (6.7, 2.5), (5.5, 1.2), (4.9, 1.0), (6.1, 1.4), (6.0, 1.6), (7.2, 2.5), (7.3, 1.8), (6.6, 1.4), (5.6, 2.0), (5.5, 1.0), (6.4, 2.2), (5.6, 1.3), (6.6, 1.3), (6.9, 2.1), (6.8, 2.1), (5.7, 1.3), (7.0, 1.4), (6.1, 1.4), (6.1, 1.8), (6.7, 1.7), (6.0, 1.5), (6.5, 1.8), (6.4, 1.5), (6.9, 1.5), (5.6, 1.3), (6.7, 1.4), (5.8, 1.9), (6.3, 1.3), (6.7, 2.1), (6.2, 2.3), (6.3, 2.4), (6.7, 1.8), (6.4, 2.3), (6.2, 1.5), (6.1, 1.4), (7.1, 2.1), (5.7, 1.0), (6.8, 1.4), (6.8, 2.3), (5.1, 1.1), (4.9, 1.7), (5.9, 1.8), (7.4, 1.9), (6.5, 2.0), (6.7, 1.5), (6.5, 2.0), (5.8, 1.0), (6.4, 2.1), (7.6, 2.1), (5.8, 2.4), (7.7, 2.2), (6.3, 1.5), (5.0, 1.0), (6.3, 1.6), (7.7, 2.3), (6.4, 1.9), (6.5, 2.2), (5.7, 1.2), (6.9, 2.3), (5.7, 1.3), (6.1, 1.2), (5.4, 1.5), (5.2, 1.4), (6.7, 2.3), (7.9, 2.0), (5.6, 1.1), (7.2, 1.8), (5.5, 1.3), (7.2, 1.6), (6.3, 2.5), (6.3, 1.8), (6.7, 2.4), (5.0, 1.0), (6.4, 1.8), (6.9, 2.3), (5.5, 1.3), (5.5, 1.1), (5.9, 1.5), (6.0, 1.5), (5.9, 1.8)]
data_y = [-1, -1, -1, -1, -1, 1, 1, -1, -1, 1, 1, -1, 1, 1, -1, 1, -1, -1, -1, 1, 1, -1, -1, -1, -1, 1, 1, -1, 1, -1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, 1, 1, -1, -1, -1, -1, 1, -1, 1, 1, 1, 1, 1, -1, -1, 1, -1, -1, 1, -1, 1, -1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, -1, -1, 1, 1, 1, -1, 1, -1, -1, -1, -1, 1, 1, -1, 1, -1, 1, 1, 1, 1, -1, 1, 1, -1, -1, -1, -1, 1]

x_train = np.array([[1, x[0], x[1]] for x in data_x])
y_train = np.array(data_y)

n_train = len(x_train)  # размер обучающей выборки
w = np.array([0.0, 0.0, 0.0])  # начальные весовые коэффициенты
nt = np.array([0.5, 0.01, 0.01])  # шаг обучения для каждого параметра w0, w1, w2
lm = 0.01  # значение параметра лямбда для вычисления скользящего экспоненциального среднего
N = 500  # число итераций алгоритма SGD

np.random.seed(0) # генерация одинаковых последовательностей псевдослучайных чисел
TP = FP = FN = TN = 0
for _ in range(N):
	k = np.random.randint(0, n_train-1)
	w = w -  nt * df(w, x_train[k], y_train[k])
    
a = lambda w, x: np.sign(np.dot(w, x))

for i in range(n_train):
	
	if a(w, x_train[i]) == 1 and y_train[i] == 1:
		TP += 1
	elif a(w, x_train[i]) == 1 and y_train[i] == -1:
		FP += 1
	elif a(w, x_train[i]) == -1 and y_train[i] == 1:
		FN += 1
	elif a(w, x_train[i]) == -1 and y_train[i] == -1:
		TN += 1

precision = TP / (TP + FP) # точность
recall = TP / (TP + FN) # полнота

print(precision, recall, TP)

0.9722222222222222 0.7 35


# Task 3

Support Vector Machine with True/False Positive/Negative with F, F-beta scores

<img src=".././photo/condition22.png" alt="photo" width="900" height="200">

In [2]:
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split

np.random.seed(0)

# исходные параметры распределений классов
r1 = 0.2
D1 = 3.0
mean1 = [2, -2]
V1 = [[D1, D1 * r1], [D1 * r1, D1]]

r2 = 0.5
D2 = 2.0
mean2 = [-1, -1]
V2 = [[D2, D2 * r2], [D2 * r2, D2]]

# моделирование обучающей выборки
N1 = 2500
N2 = 1500
x1 = np.random.multivariate_normal(mean1, V1, N1).T
x2 = np.random.multivariate_normal(mean2, V2, N2).T

data_x = np.hstack([x1, x2]).T
data_y = np.hstack([np.ones(N1) * -1, np.ones(N2)])

x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, random_state=123,test_size=0.4, shuffle=True)

clf = svm.SVC(kernel='linear')
clf.fit(x_train, y_train)

predict = clf.predict(x_test)

w_intercept = clf.intercept_[0]
w_other = clf.coef_[0]
w = np.hstack([w_intercept, w_other])
# print(w)

TN = TP = FN = FP = 0
for i in range(len(x_test)):

    if predict[i] == y_test[i] == 1:
        TP += 1
    elif predict[i] == y_test[i] == -1:
        TN += 1
    elif predict[i] == 1 and y_test[i] == -1:
        FP += 1
    elif predict[i] == -1 and y_test[i] == 1:
        FN += 1

precision = TP / (TP + FP) # the ratio of the true to all the positives of the model
recall = TP / (TP + FN) # the ratio of the true to all the positives of the general
print(precision, recall)

# F score
F = 2 * precision * recall / (precision + recall)
# F-beta score (beta = 0.5 mean precision more consider)
Fb = (1 + 1/2 ** 2) * precision * recall / (((1/2) ** 2) * precision + recall)

print(F)
print(Fb)

0.8112324492979719 0.8637873754152824
0.8366854384553499
0.8212255211623498
